In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
import math
from nltk.probability import FreqDist

# Duas maais frequentes categorias
categorias = ["",""] 
frequencia = [0,0]
for c in brown.categories():
	if frequencia[0] == 0:
		frequencia[0] = len(brown.fileids(categories=c))
		categorias[0] = c
	
	elif frequencia[1] == 0:
		frenquencia[1] = len(brown.fileids(categories=c))
		categorias[1] = c

	if len(brown.fileids(categories=c)) > frequencia[1]:
		frequencia[1] = len(brown.fileids(categories=c))
		categorias[1] = c 

	if frequencia[1] > frequencia[0]:
		aux = frequencia[0]
		frequencia[0] = frequencia[1]
		frequencia[1] = aux
		aux2 = categorias[0]
		categorias[0] = categorias[1]
		categorias[1] = aux2
print(categorias)
documentos = []
classe = []

for category in categorias:
	for fileid in brown.fileids(category):
		documentos.append(list(brown.words(fileid)))
		if  category == "learned":
			classe.append(0)
		else:
			classe.append(1)

ps = PorterStemmer()
punctuations = list(string.punctuation)
punctuations.append("''")
stop_words = set(stopwords.words("english"))
doc_processado = []
# Data completa

for x in range(0,len(documentos)):
	palavras = []
	for y in range(0,len(documentos[x])):
		if documentos[x][y] not in punctuations:
			if documentos[x][y] not in  stop_words:
				palavras.append(ps.stem(documentos[x][y]).lower())
	doc_processado.append(palavras)



#classe data
words = []
for x in range(0, len(doc_processado)):
	for i in doc_processado[x]:
		if i not in words:
			words.append(i)
			#print("Palavras: ", len(words))
		if len(words) == 16746:
			break

freqdist = []
for x in range(0,len(doc_processado)):
	freqdist.append(FreqDist(doc_processado[x]))

tf = []
for x in range(0, len(doc_processado)):
	linha = []
	for w in words:
		linha.append(freqdist[x][w])
	tf.append(linha)

frequenciaEmDocumentos = [0] * len(words)
for x in range(0,len(doc_processado)):
	for y in range(0,len(words)):
		if tf[x][y] != 0:
			frequenciaEmDocumentos[y] += 1


tf_idf = []
for x in range(0, len(doc_processado)):
	linha = []
	for y in range(0, len(words)):
		linha.append(tf[x][y] * math.log(len(doc_processado)/frequenciaEmDocumentos[y]))
	tf_idf.append(linha)


import urllib.request
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import datasets
from sklearn import tree
from random import randint
import numpy as np

data = tf_idf
classeData = classe
dataAprendizagem = []
classeAprendizagem = []

limite = len(data)
for i in range(0, int(len(data)*0.66)):
	x = randint(0,limite-1) 
	dataAprendizagem.append(data[x])
	data.remove(data[x])
	classeAprendizagem.append(classeData[x])
	classeData.remove(classeData[x])
	limite = limite - 1
X = dataAprendizagem
y = classeAprendizagem

#Nearest Neighbors http://scikit-learn.org/stable/modules/neighbors.html
clf = NearestCentroid()
clf.fit(X, y)
NearestCentroid(metric='euclidean', shrink_threshold=None)
resultNeighbors = clf.predict(data)

matrizConfusao = [[0,0],[0,0]]

learned = 0
belles_lettres = 0

for i in range(0, len(data)):
	if classeData[i] == 0:
		learned += 1
	elif classeData[i] == 1:
		belles_lettres += 1

for i in range(0,len(resultNeighbors)):
	if resultNeighbors[i] == 0:
		if classeData[i] == 0:
			matrizConfusao[0][0] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][0] += 1
	elif resultNeighbors[i] == 1:
		if classeData[i] == 0:
			matrizConfusao[0][1] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][1] += 1

accuracy = (matrizConfusao[0][0] + matrizConfusao[1][1])/(matrizConfusao[0][0]+matrizConfusao[0][1]+matrizConfusao[1][0]+matrizConfusao[1][1])
Precision = matrizConfusao[0][0]/(matrizConfusao[0][0] + matrizConfusao[0][1])
Sensitivity = matrizConfusao[0][0]/ (matrizConfusao[0][0] + matrizConfusao[1][0])
Specificity = matrizConfusao[1][1] /(matrizConfusao[0][1] + matrizConfusao[1][1])
print("\n\nMatriz de Confusão (Nearest Neighbors):")
print("					Classificação")
print("\t\t\t\tlearned\t\tbelles_lettres")
print("Classe  learned 		 %d\t\t%d" %(matrizConfusao[0][0],matrizConfusao[0][1]))
print("Real    belles_lettres		  %d\t\t%d" %(matrizConfusao[1][0],matrizConfusao[1][1]))
print("\naccuracy ", accuracy)
print("precision ", Precision)
print("sensitivity ", Sensitivity)
print("specificity ", Specificity)

#Naive Bayes http://scikit-learn.org/stable/modules/naive_bayes.html
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X, y)
resultBayes = gnb.predict(data)

matrizConfusao = [[0,0],[0,0]]

for i in range(0,len(resultBayes)):
	if resultBayes[i] == 0:
		if classeData[i] == 0:
			matrizConfusao[0][0] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][0] += 1
	elif resultBayes[i] == 1:
		if classeData[i] == 0:
			matrizConfusao[0][1] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][1] += 1

accuracy = (matrizConfusao[0][0] + matrizConfusao[1][1])/(matrizConfusao[0][0]+matrizConfusao[0][1]+matrizConfusao[1][0]+matrizConfusao[1][1])
Precision = matrizConfusao[0][0]/(matrizConfusao[0][0] + matrizConfusao[0][1])
Sensitivity = matrizConfusao[0][0]/ (matrizConfusao[0][0] + matrizConfusao[1][0])
Specificity = matrizConfusao[1][1] /(matrizConfusao[0][1] + matrizConfusao[1][1])

print("\n\nMatriz de Confusão (Naive Bayes):")
print("					Classificação")
print("\t\t\t\tlearned\t\tbelles_lettres")
print("Classe  learned 		 %d\t\t%d" %(matrizConfusao[0][0],matrizConfusao[0][1]))
print("Real    belles_lettres		  %d\t\t%d" %(matrizConfusao[1][0],matrizConfusao[1][1]))
print("\naccuracy ", accuracy)
print("precision ", Precision)
print("sensitivity ", Sensitivity)
print("specificity ", Specificity )



#Decision Trees http://scikit-learn.org/stable/modules/tree.html
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)
resultTress = clf.predict(data)

matrizConfusao = [[0,0],[0,0]]


for i in range(0,len(resultTress)):
	if resultTress[i] == 0:
		if classeData[i] == 0:
			matrizConfusao[0][0] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][0] += 1
	elif resultTress[i] == 1:
		if classeData[i] == 0:
			matrizConfusao[0][1] += 1
		elif classeData[i] == 1:
			matrizConfusao[1][1] += 1

accuracy = (matrizConfusao[0][0] + matrizConfusao[1][1])/(matrizConfusao[0][0]+matrizConfusao[0][1]+matrizConfusao[1][0]+matrizConfusao[1][1])
Precision = matrizConfusao[0][0]/(matrizConfusao[0][0] + matrizConfusao[0][1])
Sensitivity = matrizConfusao[0][0]/ (matrizConfusao[0][0] + matrizConfusao[1][0])
Specificity = matrizConfusao[1][1] /(matrizConfusao[0][1] + matrizConfusao[1][1])


print("\n\nMatriz de Confusão (Decision Trees):")
print("					Classificação")
print("\t\t\t\tlearned\t\tbelles_lettres")
print("Classe  learned 		 %d\t\t%d" %(matrizConfusao[0][0],matrizConfusao[0][1]))
print("Real    belles_lettres		  %d\t\t%d" %(matrizConfusao[1][0],matrizConfusao[1][1]))
print("\naccuracy ", accuracy)
print("precision ", Precision)
print("sensitivity ", Sensitivity)
print("specificity ", Specificity )

['learned', 'belles_lettres']


Matriz de Confusão (Nearest Neighbors):
					Classificação
				learned		belles_lettres
Classe  learned 		 15		12
Real    belles_lettres		  0		26

accuracy  0.7735849056603774
precision  0.5555555555555556
sensitivity  1.0
specificity  0.6842105263157895


Matriz de Confusão (Naive Bayes):
					Classificação
				learned		belles_lettres
Classe  learned 		 19		8
Real    belles_lettres		  3		23

accuracy  0.7924528301886793
precision  0.7037037037037037
sensitivity  0.8636363636363636
specificity  0.7419354838709677


Matriz de Confusão (Decision Trees):
					Classificação
				learned		belles_lettres
Classe  learned 		 24		3
Real    belles_lettres		  9		17

accuracy  0.7735849056603774
precision  0.8888888888888888
sensitivity  0.7272727272727273
specificity  0.85


In [ ]:
Ao avaliar os valores da métricas, pode-se concluir que para esse problema, o algoritmo de Naive Bayes apresenta bons resultados. Pois teve melhor taxa de acerto, boa precisão, sensibilidade e especidade.
